In [ ]:
from __future__ import division, print_function, absolute_import
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

from keras.applications.mobilenet import decode_predictions as mobilenet_decode_predictions
from keras.applications.mobilenet import preprocess_input as mobilenet_preprocess_input

# Helper functions
import helper
from attack import PixelAttacker

%matplotlib inline
matplotlib.style.use('ggplot')
np.random.seed(100)

In [ ]:
# Load the Model
model = keras.applications.MobileNet()

In [ ]:
# Make a dictionary that maps the name of the class (in like real english) to the index
with open('data/imagenet_classes.pkl', 'rb') as f:
    class_names = pickle.load(f)
word_to_class = {w:i for i,w in enumerate(class_names)}

In [ ]:
# Load Some Test Images
fnames = (
    'data/ILSVRC2012_val_00001036.JPEG', 
    'data/ILSVRC2012_val_00000655.JPEG', 
    'data/ILSVRC2012_val_00009401.JPEG', 
    'data/ILSVRC2012_val_00000962.JPEG', 
    'data/ILSVRC2012_val_00000911.JPEG', 
    'data/ILSVRC2012_val_00000329.JPEG', 
    'data/ILSVRC2012_val_00002138.JPEG', 
    'data/ILSVRC2012_val_00000293.JPEG'
)
names = (
    'wallaby, brush kangaroo', 
    'wallaby, brush kangaroo', 
    'water ouzel, dipper', 
    'water ouzel, dipper', 
    'cock', 
    'cock', 
    'tench, Tinca tinca', 
    'tench, Tinca tinca'
)

images = []
labels = []

for fname,name in zip(fnames,names):
    img = load_img(fname, target_size=(224, 224))
    x = img_to_array(img, data_format='channels_last')
    images.append(x)
    labels.append([word_to_class[name]])
    
images = np.array(images)
labels = np.array(labels)

In [ ]:
# Normalize ImageNet Images
#processed_images = preprocess_input(images.copy(), data_format='channels_last')

processed_images = mobilenet_preprocess_input(images.copy(), data_format='channels_last')
starting_images = processed_images.copy()
print("It is import to use the correct preprocessing function for the model")

print("Processed Images shape {}".format(processed_images.shape))

In [ ]:
# Single Image Prediction
target_imd_idx = 0

prediction = model.predict(np.expand_dims(processed_images[target_imd_idx, ], axis=0))
helper.plot_image(images[target_imd_idx,])
decode_predictions(prediction)

In [ ]:
# Set up the attack
models = [model]

# The data set we will try to attack
test_tuple = (processed_images, labels)

attacker = PixelAttacker(models, test_tuple, class_names, dimensions=(224,224))

In [ ]:
result = attacker.attack(target_imd_idx, model, maxiter=20, verbose=True)

In [ ]:
# Decode the Results
print("Was Attack successful {}".format(result[5]))
print("True Class ({}:{}), Final Predicted class ({}:{})".format(
    result[3], class_names[result[3]], result[4], class_names[result[4]]))
print("Drop in confidence in true class {}".format(result[6]))


In [ ]:
# Show the difference between start and stop image
helper.plot_image(images[target_imd_idx,])
plt.title('Post Processing Image')

print(np.max(images[target_imd_idx,]), np.min(images[target_imd_idx,]))

